![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Lab 1: Vector Search with RedisVL
---

## Introduction 

Let’s get started with the first lab of the Redis AI workshop. In this lab, you will get hands-on experience using the [RedisVL](https://github.com/redis/redis-vl-python) library to perform vector search on a JSON dataset of popular movies.

Before we dive in, let's cover the structure of the lab. This lab is broken out into the following five sections:

1. Introduction (📍 You are here )
2. Set up Redis and import the data
3. Vectorize the data
4. Perform vector search techniques
5. Wrap Up

You’ll go from setting up the environment all the way to using a variety of vector search techniques. This will help you build the foundation for performing vector search in real-world AI applications. 

This lab will also have a few tasks (denoted with a 📌) for you to get hands on with the code. When applicable, tasks come with a solution code to compare too.

Additionally, if you ever want to view the database at any point during the lab, you can return to the lab documentation page and open Redis Insight.

Now that you know what to expect, let's get started.


## Set up a Redis connection and import the data

Before performing any kind of search, we will need to set up Redis and load some data to search. For this lab, we will be using a dataset of movies with the following attributes: 

- title
- rating
- description
- genre

Let's start by establishing a connection to the local Redis instance, which is already running in this notebook.

### Connect to Redis

To get started, we need to connect to Redis using environment variables that have already been set up for you. These include the host, port, and password, which we’ll use to build a connection URL for the Redis client.

The code below initializes the connection variables and creates a `redis_connection` object. We verify the connection using `.ping()`.

Take a moment to review the code, then run the code block. If the connection is successful, it will return `True`.

In [1]:
import os
import warnings
from redis import Redis

warnings.filterwarnings('ignore')

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")

REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

redis_connection = Redis.from_url(REDIS_URL)
redis_connection.ping()

True

Now that the connection to Redis is established, let's move on to importing the movie data.

### Import the movie data

The movie dataset we'll be using is stored in `movies.json`, located in the `/resources` directory.

Instead of loading data directly into Redis, we'll first import it into a Pandas DataFrame. This gives us a local copy to work with while we convert the data into vector embeddings for search.

Take a moment to review the code, then run the code block. If it is successful, you'll see a preview table and a count of the movies loaded.

⚠️ **Note:** The code block below may take some time to run. Please be patient as it completes, and observe the results.

In [2]:
import pandas as pd

df = pd.read_json("resources/movies.json")
print("Loaded", len(df), "movie entries")

df.head()

Loaded 55 movie entries


,id,title,genre,rating,description
0,1,Explosive Pursuit,action,7,A daring cop chases a notorious criminal acros...
1,2,Skyfall,action,8,James Bond returns to track down a dangerous n...
2,3,Fast & Furious 9,action,6,Dom and his crew face off against a high-tech ...
3,4,Black Widow,action,7,Natasha Romanoff confronts her dark past and f...
4,5,John Wick,action,8,A retired hitman seeks vengeance against those...


Next, lets prepare the data we imported for vector search.

## Vectorize the data

In this section, we'll vectorize the data. First, we’ll take the description field from each JSON document and transform it into a vector embedding. We’ll store that embedding as an additional property in the document, alongside its other fields. Then, we’ll create a Redis search index that includes all document fields (including the embedding). In practice, you only need to index the fields that will be queried. In this case, that happens to be all of them.

First, we’ll transform our raw JSON data into vector embeddings, and then we’ll take those embeddings and index them in Redis, making them searchable. 

By the end, we'll have gone from raw data to a fully indexed, queryable dataset — ready for vector search. This setup will mirror the retrieval foundation we'll rely on in the next lab, where we’ll build a full RAG workflow on top of what we create here.

### A recap on vector embeddings

Let’s revisit a core concept behind most LLM-powered applications: vector embeddings.

Embeddings are high-dimensional vectors — like `[0.1316, 0.7251, 0.4055, ...]` — that represent the semantic content of text in a format that machine learning models can work with. This lets us compare text based on contextual similarity, even when they don’t share the same words.

You'll use RedisVL to generate and store embeddings so we can search our dataset by semantic relevance — not just keywords.

### Creating embeddings using RedisVL

The RedisVL library provides helpful methods to create and manage vector embeddings, making it easier to integrate them into AI workflows. Below, we’ll walk through how to embed a list of movie descriptions using a transformer model and store the results in a DataFrame for later indexing.

There are two key classes that we will use from RedisVL: 

1. `HFTextVectorizer`: This is a wrapper around Hugging Face transformer models that makes it easy to generate vector embeddings.

2. `EmbeddingsCache`: This class helps store and reuse embeddings using Redis as a cache.

Take a moment to review the code, then run the code block. The arguments provided are self-explanatory, but optionally, you can explore a more detailed breakdown in the dropdown below.

⚠️ **Note:** The code block below may take some time to run. Please be patient as it completes, and observe the results.

In [3]:
from redisvl.utils.vectorize import HFTextVectorizer
from redisvl.extensions.cache.embeddings import EmbeddingsCache

os.environ["TOKENIZERS_PARALLELISM"] = "false"


hf = HFTextVectorizer(
    model="sentence-transformers/all-MiniLM-L6-v2",
    cache=EmbeddingsCache(
        name="embedcache",
        ttl=600,
        redis_client=redis_connection,
    )
)

17:57:58 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: cpu
17:57:58 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<details>
<summary> (Optional) What does the <code>"TOKENIZERS_PARALLELISM"</code> setting do? </summary>
<br>
This setting is used to clean up the output we receive. It suppresses a warning from the Hugging Face transformers library related to tokenizer parallelism. By default, tokenizers may try to use multiple threads, which can cause warnings or unnecessary overhead in a notebook environment.
</details>

<details>
<summary> (Optional) HFTextVectorizer parameter breakdown </summary>
<br>
For <code>HFTextVectorizer</code> we provided two arguments: 

- model: The name of a sentence transformer model from Hugging Face. In our case, we’re going to use "sentence-transformers/all-MiniLM-L6-v2", a fast and lightweight model ideal for semantic search.

- cache): An optional argument that assigns an instance of `EmbeddingsCache` that helps avoid recomputing embeddings for repeated inputs.

For <code>EmbeddingsCache</code> we provided two arguments: 

- name: A key prefix to use in Redis for the cache

- ttl: An optional argument that sets how long the cached embeddings should live in Redis (in seconds)

- redis_client: The Redis client used to store and retrieve vectors.

A more detailed list of all parameters can be found in the RedisVL documentation for [HFTextVectorizer](https://docs.redisvl.com/en/latest/api/vectorizer.html#hftextvectorizer) and [EmbeddingsCache](https://docs.redisvl.com/en/latest/api/cache.html#embeddingscache).

</details>

Now, let's use the vectorizer on the movie dataset. Review the code below, run it, and observe the results.

In [4]:
df["vector"] = hf.embed_many(df["description"].tolist(), as_buffer=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The code above took the description column of the DataFrame, passed the text to the `embed_many()` method, and generated vector embeddings in batches, storing the resulting vectors—encoded as compact binary buffers (specified by `as_buffer=True`), in a new column (in the same DataFrame) called "vector".

<details>
<summary>(Optional) What is a compact binary buffer?</summary>
<br>
A compact binary buffer is a space-efficient way to store data. In this case, vector embeddings are stored as raw bytes instead of a list of floating-point numbers (e.g., <code> b'\x00\x00\x80?\x9a\x99\x19@...'</code> instead of <code>[1.0, 6.4, ...]</code>).

The buffer uses the `dtype` argument specified in the vectorizer, which defaults to "float32". Since we wanted to keep it as the default, we didn’t need to set it explicitly in our call above.

</details>

Preview the new column by running the following code block:

In [5]:
df.head()

,id,title,genre,rating,description,vector
0,1,Explosive Pursuit,action,7,A daring cop chases a notorious criminal acros...,b'\x9ef|=\xc8a\n;\xde\x91\xb7; \xcb~\xbd*e\xce...
1,2,Skyfall,action,8,James Bond returns to track down a dangerous n...,b'\x9eD\x9e\xbd?\x9b\x89\xbc\xbc\x16\x95\xbc\x...
2,3,Fast & Furious 9,action,6,Dom and his crew face off against a high-tech ...,"b'$\xa5\xc7\xbc\xfc,\xa2=L\x19H\xbc=\xc6t\xbd\..."
3,4,Black Widow,action,7,Natasha Romanoff confronts her dark past and f...,b't\xeb\x85\xbd\x04\xcdo\xbd\xb9\xe8\xc2\xbb;\...
4,5,John Wick,action,8,A retired hitman seeks vengeance against those...,b'4<x\xbb\x02/\xc5=\xff\x86:;\xc7\xd0\x94<\xfc...


Now that we have our vector embeddings, let's move on to creating an index and schema to enable the ability to search them.

### Schema creation overview

Recall that in core Redis, the definition of a schema and the creation of an index are performed together using the [`FT.CREATE`](https://redis.io/docs/latest/commands/ft.create/) command.

In RedisVL, the process is split into two parts: defining the schema in code and then creating the index.

The schema can be created by using the [`IndexSchema`](https://docs.redisvl.com/en/latest/api/schema.html#indexschema) class, which lets you define the schema from either a YAML file or a Python dictionary. In this lab, we’ll define one using a dictionary by leveraging the `.from_dict()` method. 

The schema has two main parts that must be defined:

1. An index section, which configures index-specific settings such as the index name, key prefix, and whether documents are stored as Hashes or JSON. 

2. A fields list, which defines what fields should be indexed and how they should be searched. Each field has a name, a type (e.g., text, tag, numeric, or vector), and any required/optional attributes. A full list of supported field types and attributes can be found in the [RedisVL documentation](https://docs.redisvl.com/en/latest/api/schema.html#supported-field-types-and-attributes). 

You'll build the full schema for the movie data shortly, but to familiarize yourself with the syntax, examine the partial example below before proceeding.

```python 

schema = IndexSchema.from_dict({
  "index": {
    "name": "movies",
    "prefix": "...",
    "storage_type": "..."
  },
  "fields": [
    {
        "name": "title",
        "type": "text",
        "attrs": {
          ...
        }
    },
    {
      ...
    }
  ]
})

```

### Define the Index Schema

It's time to create the schema. Complete the task below.

📌 **Task 1: Create the schema**

We've provided the starting code for the movie data schema. Your task is to complete it using the field definitions listed below.

Create a schema named `"movies"` with:
- A prefix of `"movies"`
- A storage type of `"hash"`

Include the following fields:

| Field Name   | Type      | Attributes                                                                 |
|--------------|-----------|---------------------------------------------------------------------------|
| `title`      | `text`    | _none_                                                                     |
| `description`| `text`    | _none_                                                                     |
| `genre`      | `tag`     | `sortable: True`                                                           |
| `rating`     | `numeric` | `sortable: True`                                                           |
| `vector`     | `vector`  | `dims: 384`  <br> `distance_metric: "cosine"` <br> `algorithm: "flat"` <br> `datatype: "float32"` |


Remember to run the code block once the schema is complete. The solution code is posted below if you need help or want to compare your answer.

In [8]:
from redisvl.schema import IndexSchema

index_name = "movies"

schema = IndexSchema.from_dict({
    "index": {
        "name": "movies",
        "prefix": "movies",
        "storage_type": "hash"

    },
    "fields": [
        # Add the required fields below
        {
            "name": "title",
            "type": "text",       
        },
        {
            "name": "description",
            "type": "text",       
        },
        {
            "name": "genre",
            "type": "tag",
            "attrs": {
                "sortable": True
            }
        },
        {
            "name": "rating",
            "type": "numeric",
            "attrs": {
                "sortable": True
            }
        },
        {
            "name": "vector",
            "type": "vector",
            "attrs": {
                "dims": 384,
                "distance_metric": "cosine",
                "algorithm": "flat",
                "datatype": "float32"
            }
        },

    ]
})

print('Schema created:', schema)

Schema created: index=IndexInfo(name='movies', prefix='movies', key_separator=':', storage_type=<StorageType.HASH: 'hash'>) fields={'title': TextField(name='title', type=<FieldTypes.TEXT: 'text'>, path=None, attrs=TextFieldAttributes(sortable=False, index_missing=False, weight=1, no_stem=False, withsuffixtrie=False, phonetic_matcher=None, index_empty=False)), 'description': TextField(name='description', type=<FieldTypes.TEXT: 'text'>, path=None, attrs=TextFieldAttributes(sortable=False, index_missing=False, weight=1, no_stem=False, withsuffixtrie=False, phonetic_matcher=None, index_empty=False)), 'genre': TagField(name='genre', type=<FieldTypes.TAG: 'tag'>, path=None, attrs=TagFieldAttributes(sortable=True, index_missing=False, separator=',', case_sensitive=False, withsuffixtrie=False, index_empty=False)), 'rating': NumericField(name='rating', type=<FieldTypes.NUMERIC: 'numeric'>, path=None, attrs=NumericFieldAttributes(sortable=True, index_missing=False)), 'vector': FlatVectorField(na

<details>
<summary> 🔒 Schema Solution Code </summary>
<br>

```python

from redisvl.schema import IndexSchema

index_name = "movies"

schema = IndexSchema.from_dict({
    "index": {
        "name": "movies",
        "prefix": "movies",
        "storage_type": "hash"
    },
    "fields": [
        {
            "name": "title",
            "type": "text"
        },
        {
            "name": "description",
            "type": "text"
        },
        {
            "name": "genre",
            "type": "tag",
            "attrs": {
                "sortable": True
            }
        },
        {
            "name": "rating",
            "type": "numeric",
            "attrs": {
                "sortable": True
            }
        },
        {
            "name": "vector",
            "type": "vector",
            "attrs": {
                "dims": 384,
                "distance_metric": "cosine",
                "algorithm": "flat",
                "datatype": "float32"
            }
        }
    ]
})

print('Schema created:', schema)

```

</details>

### Add an index

Along with the schema, we need to create the index. To do so, we can use the `SearchIndex` class, which has a `.create()` method. The class takes two required arguments:

- `schema`: The index schema that you want to use.

- `client`: A Redis client instance.

Run the code block below. There will be no output since we are creating the index, but we will observe it in the next step.

In [9]:
from redisvl.index import SearchIndex

index = SearchIndex(schema, redis_connection)
index.create(overwrite=True, drop=True)

       
> **🚨 Extreme Caution:** It is important to be aware when using the `.create()` method, `overwrite=True` overwrites an index in Redis
> without dropping associated data. However, setting `drop=True` will delete both the index AND **all associated documents** (clean slate). `drop=True` will only take effect when `overwrite=True` is also set.
> 
> Be very careful when setting this, as a mistake may cause data loss.

We can use [`rvl`](https://docs.redisvl.com/en/latest/overview/cli.html), the CLI built into RedisVL, to confirm that the index was created. Run the code below to view and confirm the movies index was created.

In [10]:
!rvl index info -i movies -u {REDIS_URL}



Index Information:
╭───────────────┬───────────────┬───────────────┬───────────────┬───────────────╮
│ Index Name    │ Storage Type  │ Prefixes      │ Index Options │ Indexing      │
├───────────────┼───────────────┼───────────────┼───────────────┼───────────────┤
| movies        | HASH          | ['movies']    | []            | 0             |
╰───────────────┴───────────────┴───────────────┴───────────────┴───────────────╯
Index Fields:
╭─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────╮
│ Name            │ Attribute       │ Type            │ Field Option    │ Option Value    │ Field Option    │ Option Value    │ Field Option    │ Option Value    │ Field Option    │ Option Value    │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼──────────

### Populate the index

Now, we can load our data into Redis using `.load()` method. This method has one required argument (and several optional ones) called `data` which takes an iterable to load (such as our DataFrame rows).  When called, it will write each record to Redis, 
and construct keys using the index prefix, ensuring the data is indexed automatically according to the schema.

Run the following code block to load the records:

In [11]:
index.load(df.to_dict(orient="records"))

['movies:01K980WTCFM74Y51V1ZR7EGA00',
 'movies:01K980WTCFM74Y51V1ZR7EGA01',
 'movies:01K980WTCFM74Y51V1ZR7EGA02',
 'movies:01K980WTCFM74Y51V1ZR7EGA03',
 'movies:01K980WTCFM74Y51V1ZR7EGA04',
 'movies:01K980WTCFM74Y51V1ZR7EGA05',
 'movies:01K980WTCFM74Y51V1ZR7EGA06',
 'movies:01K980WTCFM74Y51V1ZR7EGA07',
 'movies:01K980WTCFM74Y51V1ZR7EGA08',
 'movies:01K980WTCFM74Y51V1ZR7EGA09',
 'movies:01K980WTCFM74Y51V1ZR7EGA0A',
 'movies:01K980WTCFM74Y51V1ZR7EGA0B',
 'movies:01K980WTCFM74Y51V1ZR7EGA0C',
 'movies:01K980WTCFM74Y51V1ZR7EGA0D',
 'movies:01K980WTCFM74Y51V1ZR7EGA0E',
 'movies:01K980WTCFM74Y51V1ZR7EGA0F',
 'movies:01K980WTCFM74Y51V1ZR7EGA0G',
 'movies:01K980WTCFM74Y51V1ZR7EGA0H',
 'movies:01K980WTCFM74Y51V1ZR7EGA0J',
 'movies:01K980WTCFM74Y51V1ZR7EGA0K',
 'movies:01K980WTCFM74Y51V1ZR7EGA0M',
 'movies:01K980WTCFM74Y51V1ZR7EGA0N',
 'movies:01K980WTCFM74Y51V1ZR7EGA0P',
 'movies:01K980WTCFM74Y51V1ZR7EGA0Q',
 'movies:01K980WTCFM74Y51V1ZR7EGA0R',
 'movies:01K980WTCFM74Y51V1ZR7EGA0S',
 'movies:01K

<details>
<summary>(Optional) What does <code>orient="records"</code> do? </summary>
<br>
The orient property determines the type of the values of the dictionary. Passing <code>"records"</code> as the value for the orient property makes it so the data has the following structure:

  - Each dictionary represents one row (or document)
  - The keys are the column names, and the values are the corresponding cell values

For example:

```python

[ 
  {"title": "Finding Nemo", "genre": "Animation", "rating": 8.1, "vector": [...]},
  ...
]

```

The full list of acceptable values for the orient property can be found in the <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html#pandas.DataFrame.to_dict">Pandas DataFrame documentation.</a>
</details>

📌 **Task 2: Confirm the data**  

Confirm the data by fetching one of the movies in the database. Grab any of the keys that were loaded above (e.g., `'movies:01K0791...'`) and plug it into the code below. Then run it to confirm that it is returned and has a vector field. Alternatively, the data can be viewed in Redis Insight.

In [13]:
# Ex. redis_connection.hgetall("movies:01K07...")

redis_connection.hgetall("movies:01K980WTCG07K1WSANC2MB6M6Q")

{b'description': b'A detective investigates the death of a patriarch of an eccentric, combative family.',
 b'vector': b'\xff{4\xbd\x1c\xca\xf5\xbaW~\xae\xbd\xe3\x95\xf4\xbb\xcb\xcf?<`_7=\xa5N\xb7;\xeb\xde\x07=h\x87\x9c<2\xfcw=\xbc\x15\xa5=\xe8\xe3\x8b\xbc\xf0S <\xa0\xa9\n\xbb\xc5\xfa\xe2\xbd\x1b\xe5\xd7;*3\'\xbd\x0bh\xe9<\x96I\x94\xbc\xa3\x7fT=QE<\xbd\xa1)\x84\xbd\x15\x8c\xae=\xaf\x1b\xb8\xbd\xe6\xf3\x81\xbd\xc0\xe5\x97\xbb^\xe3\xf9<Y\xad=\xbd\xea\xe8Y\xbd\xdb\xc8\x02=5\x84\x7f;\xe3\x9e-<Vp\xf2=\xae\x88\x9a=Q[\xa9<\xa3\x93\'\xbd\xa6\x8b\x1a=\xb9\x16W<n}\x00=\xde\x90)=\xa5\x0f\x97\xbc\x03\xa9\x91<\'\x02\x7f=k\x84\xcf\xbc\x88J\xdd\xbb@}?\xbd\x10N\'\xbd\xd2\x01G=\x8f\xfc\x8f=\x16\xe8\x0b\xbd\x93_\x83\xbd\x9cY\xc9\xbdh\xc8\xf3<\xc3\x07@\xbc\xb4\xfe\xae\xbc\x81\xa4\x0e\xbd\xc9\xfcI=\x11\xeb\xe8<r[\x03\xbd\xcdw\x82<s\xf4L=\xeb\xda\xa7<\x9c\x960\xbc7w\xad<f\xb1E=\xdc\x8dS\xbc\xaf#\x08=g=I\xbd&\xd6\x8c=O\x83\x96<$\xe6-\xbc\n\xc6\x95;Wh\x94\xbd\xbb\x05\xa9=\x1c\x82\x08\xbeP\x8f+=\xb2lw<\xe2\x89

With that complete, we have now successfully stored and indexed the data in Redis. We're now ready to explore how to search it. 

## Search techniques

In this section, we’ll perform a variety of search techniques on the data we loaded. This includes:

- KNN (K-Nearest Neighbors) search
- Vector search with filters
- Range queries
- Hybrid search

At this stage, we're not using a large language model (LLM) yet — just pure vector search. However, in the next lab, we’ll build on these capabilities to retrieve relevant content that augments the AI-generated response, forming the foundation of RAG.

### KNN search

Let's start with the most basic form of vector search that implements the K-Nearest Neighbors (KNN) algorithm. This type of search finds the items in a dataset that are closest in meaning to a given query, even if they are not all relevant.

First, we'll need to embed a specific user query. For now, we'll hardcode this and use `"High tech and action packed movie"`. We can leverage the vectorizer we created earlier (`hf`) and call the `.embed()` method (earlier we used the alternative `.embed_many()`).

Run the following code block:

In [14]:
user_query = "High tech and action packed movie"

embedded_user_query = hf.embed(user_query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Now, we can perform the actual search. To do so, we can use the RedisVL [`VectorQuery`](https://docs.redisvl.com/en/latest/api/query.html#) class. Below, we've defined a query that will get the three closest matches and return the `"title"` and `"genre"` fields.

Run the code block and observe the search query string it returns.

In [15]:
from redisvl.query import VectorQuery

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "genre"],
    return_score=True,
)

print("Vector query:", vec_query)

Vector query: *=>[KNN 3 @vector $vector AS vector_distance] RETURN 3 title genre vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 3


<details>
<summary> (Optional) VectorQuery parameter breakdown </summary>
<br>

Above, we defined VectorQuery with the following parameters: 

- vector: The embedded version of the user's query (we generated this in the previous step, and stored it in the `embedded_user_query` variable).

- vector_field_name: The name of the vector field in the index (we used `"vector"` in the schema).

- num_results: The number of closest matches to return — this is the "K" in KNN.

- return_fields: An optional argument that lists the document fields to return from Redis (e.g., "title", "genre"). By default, it will return all the fields.

- return_score: An optional argument that includes the similarity score for each result (lower means more similar). This defaults to `False` if not set and will not return the score.

The full list of available parameters can be found in the [RedisVL VectorQuery class API documentation](https://docs.redisvl.com/en/latest/api/query.html#vectorquery).
</details>

Notice that the resulting `vec_query` is a low-level Redis query expression. This is because, in the next step, RedisVL will use the `FT.SEARCH` command under the hood to execute it.

Now that we’ve constructed the query object, we can pass it to `index.query()` to perform the search. Wrapping the results with `pd.DataFrame()` will format the result as a readable table. Run the code below to see it in action!

In [16]:
result = index.query(vec_query)
pd.DataFrame(result)

,id,vector_distance,title,genre
0,movies:01K980WTCFM74Y51V1ZR7EGA02,0.649737894535,Fast & Furious 9,action
1,movies:01K980WTCFM74Y51V1ZR7EGA13,0.731304645538,No Country for Old Men,Crime
2,movies:01K980WTCFM74Y51V1ZR7EGA05,0.763234972954,Mad Max: Fury Road,action


We can see that the query is returning some results for "high tech and action packed" movies. 

📌 **Task 3: Experiment with the query**  

Take a moment to change the user query code block above and rerun the search. Here are some suggestions to try: 

- Wholesome animated story about friendship
- I want something that feels like Pixar

⚠️ **Note:** After changing the query, you'll need to rerun the three most recent code blocks above for the new query to run.

When you're finished, move on to the next search technique.

### Vector search with filters

Redis allows you to combine filter searches on fields within the index object which helps create more specific searches. There are a variety of [filters](https://docs.redisvl.com/en/latest/api/filter.html#) for text, number, tag, and geo fields. 

For example, say we want to search for the top three movies specifically in the "action" genre. Since the field is a tag (we specified it as a tag in the index schema), we can use the `Tag` class combined with the `.set_filter()` method to filter by its value. 

Note that the `.set_filter()` method is a quick way to set the filter expression parameter of the vector query we defined earlier without rewriting it. 

Run the code block below to establish and query by the tag. Assuming the query is still `"High tech and action packed movie"`, the result should contain only the movies that are of the "action" genre. If you changed the query in the earlier code block, the result will be different. 

In [ ]:
from redisvl.query.filter import Tag

tag_filter = Tag("genre") == "action"

vec_query.set_filter(tag_filter)

result=index.query(vec_query)
pd.DataFrame(result)

📌 **Task 4: Experiment with the query**  

Take a moment to change the genre to "comedy" and rerun the code block above to observe the difference.

When you're finished, move on to the next search technique.

### Vector search with text filter

Another example of a common filter is a text filter. For example, let's say we wanted to search for movies that directly mention "criminal mastermind" in the description. We can do so using the `Text` class. 

Take a look at the implementation below, and then run the code to see the result.

In [ ]:
from redisvl.query.filter import Text

text_filter = Text("description") % "criminal mastermind"

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True,
    filter_expression=text_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

Notice that above, we set the `filter_expression` directly when we created `vec_query` again. This is an alternative to using the `.set_filter()` method.

📌 **Task 5: Experiment with the query**  

Take a moment to change the text_filter to "family" and rerun the code block above to observe the difference.

When you're finished, move on to the next search technique.

### Multiple filters

In addition to using individual filters, we can also filter with a combination of filters. This can't be accomplished by calling `.set_filter()` multiple times, because it does not merge filters; rather, it overwrites any previous filter expression. 

Instead, the combined filter needs to be created ahead of time using logical operators like `&` (AND) or `|` (OR), and then passed once to `.set_filter()` or to the query when it is defined.

For example, let's say we wanted to do both the tag filter from above as well as a filter for movies with a rating of 7 or above. Examine and then run the following code block:

In [ ]:
from redisvl.query.filter import Num

num_filter = Num("rating") >= 7
tag_filter = Tag("genre") == "action"

combined_filter = tag_filter & num_filter

# rebuild vector query
vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre"],
    return_score=True,
    filter_expression=combined_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

In the above code, we are using a new filter for numeric values ("rating") with the `Num` method provided by RedisVL. Then, we combine the filters using the `&` syntax and pass it to the vector query. This will allow us to filter for movies that are of the "action" genre but also have a rating of 7 or above.

RedisVL provides a variety of other ways to search for text, such as wildcard matching and fuzzy matching. All these examples and more can be found in the RedisVL [ documentation on text filters](https://docs.redisvl.com/en/latest/user_guide/02_hybrid_queries.html#text-filters).

📌 **Task 6: Experiment with the query**  

Take a moment to change the filters around for genre and rating. We recommend searching for "comedy" movies with a rating of 8 or higher. Rerun the code block above to observe the difference.

When you're finished, move on to the next search technique.

### Range queries

So far, we've used K-Nearest Neighbors (KNN) search, which always returns the top N results, even if some aren't very similar. Range queries give us more control by letting us define a cutoff for what counts as "similar enough."

Instead of asking, "What are the closest 5 documents?" a range query lets us say, "Only return documents within a certain distance."

This is useful when you care more about quality than quantity and don’t want to include loosely related results. This distance is defined via a `distance_threshold` parameter defined on a `RangeQuery` class instance. 

Examine and then run the range query defined in the code block below.

In [ ]:
from redisvl.query import RangeQuery

user_query = "Family friendly fantasy movies"

embedded_user_query = hf.embed(user_query)

range_query = RangeQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    return_fields=["title", "rating", "genre"],
    return_score=True,
    distance_threshold=0.8  # find all items with a semantic distance of less than 0.8
)

result = index.query(range_query)
pd.DataFrame(result)


<details>
<summary> (Optional) How does distance threshold work? </summary>
<br>

Distance is interpreted using the value set for `"distance_metric"` via the index schema. In ours, we defined it as `"cosine"`. 

Cosine distance is in the range of 0 to 2 which can be normalized (to be between 0 and 1) if you set an optional `normalize_vector_distance` parameter to `True`. The cosine function isn't linear, but as a quick estimate, a value of `0.8` is roughly an 80% miss (i.e. Anything closer than 80% of the maximum possible distance).

</details>

📌 **Task 7: Experiment with the query**  

Try lowering the threshold to 0.73 to make the filter stricter and observe the results. 


### Hybrid search

So far, we’ve explored vector search, which matches based on meaning, and we know that Redis supports searching by text as well (both via a filter and a completely separate [TextQuery](https://docs.redisvl.com/en/latest/api/query.html#textquery) class for text-only queries, which do not invoke vector similarity). However, what if we want the best of both? That’s where hybrid search comes in.

Hybrid search allows you to combine full-text search with vector similarity to improve relevance and precision. 

For example, a hybrid query for "movies about artificial intelligence" might return:

- A lexical match: “Artificial Intelligence: AI” (direct keyword match)
- A Semantic match: “Ex Machina” or “Her” — movies that don’t use the exact phrase but are about the requested topic.

With hybrid search, both types of results appear—sorted by how well they match your intent and keywords.

The great news is that RedisVL has a built-in `HybridQuery` class to help run these types of queries. It takes both a text input and a query vector, and Redis will return results that reflect both signals. 

Take a look at the example hybrid query below, run the code block to establish the query object, and observe the query it creates.

In [ ]:
from redisvl.query import HybridQuery

hybrid_query = HybridQuery(
    text=user_query,
    text_field_name="description",
    text_scorer="BM25",
    vector=embedded_user_query,
    vector_field_name="vector",
    alpha=0.7,
    num_results=20,
    return_fields=["title", "description"],
)

print("Hybrid query object:", hybrid_query)

Let’s break down the new arguments introduced here:

- `text`: This is the raw keyword or phrase to use for full-text search. It will be scored using BM25. We'll simply use the query we defined earlier for "Family friendly fantasy movies" (or whatever the query was changed to when you were experimenting).

- `text_field_name`: The name of the field in your documents to apply full-text search to — in our case, "description".

- `text_scorer`: This defines how the full-text score is calculated. You must enter one of the supported methods from the RedisVL documentation (BM25 or TFIDF). The one in this example, BM25, is the standard ranking algorithm used in search engines.

- `alpha`:  Tells the search engine how much weight to give the vector similarity vs. the text similarity. Documents will be scored as: `hybrid_score = (alpha) * vector_score + (1-alpha) * text_score`. Higher alpha values favor semantic similarity, while lower values favor keyword matching.

Everything else works the same as the `VectorQuery`.

Now, let's run the hybrid query below and see the results. It should return the top 4 matching documents and show how each one scored in:

- vector similarity (the vector score)

- text score

- final hybrid score (a weighted combo)

In [ ]:
result = index.query(hybrid_query)[:4]
pd.DataFrame(result)[["title", "vector_similarity", "text_score", "hybrid_score"]]

## Wrap Up 🏁 

Great job! You've completed Lab 1: Vector Search with RedisVL.

In this lab, you learned how to:

- Connect to a Redis instance and used Pandas to read and parse a JSON document
- Use RedisVL to generate vector embeddings for JSON data
- Define and create a Redis schema and index
- Index data in Redis
- Perform a variety of search techniques, including:
    - K-Nearest Neighbors (KNN) vector search
    - Vector search with tag, text, and numeric filters
    - Range queries with distance thresholds
    - Hybrid search combining full-text and vector similarity

These are the foundational skills for building Retrieval-Augmented Generation (RAG) applications and semantic search systems.

In the next lab, you'll build on these concepts to integrate LLMs and create a production-ready RAG chatbot using Redis as your vector database.

Feel free to experiment further with the queries and filters, or revisit any section for practice.